In [2]:
import kaggle as kg
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical

from keras.layers import Input, Dense, BatchNormalization
from keras.models import Model
from keras.optimizers import SGD

In [5]:
os.environ["KAGGLE_USERNAME"] = "thasinkhan"
os.environ["KAGGLE_KEY"] = "d0e58b5e1972f61220788d36914159c2"

In [6]:
kg.api.authenticate()


In [7]:
kg.api.dataset_download_files(dataset="medahmedkrichen/devanagari-handwritten-character-datase",
                              path="dataset",unzip=True)

Dataset URL: https://www.kaggle.com/datasets/medahmedkrichen/devanagari-handwritten-character-datase


In [8]:
def train_test_df(path):

    img_path = list()
    img_label = list()

    for single_class_dir_path in pathlib.Path(path).glob("*"):

        for single_class_img_path in pathlib.Path(single_class_dir_path).glob("*.png"):

            img_path.append(str(single_class_img_path))
            #print(str(single_class_img_path).split("/")[-2].split("_")[-1])
            img_label.append(str(single_class_img_path).split("/")[-2].split("_")[-1])

    return pd.DataFrame(data={"img_path":img_path,"label":img_label})    

In [9]:
train_path = "dataset/DevanagariHandwrittenCharacterDataset/Train"
test_path = "dataset/DevanagariHandwrittenCharacterDataset/Test"

In [10]:
training_data = train_test_df(train_path)
testing_data = train_test_df(test_path)

In [11]:
training_data.shape,testing_data.shape

((78200, 2), (13800, 2))

In [12]:
training_data.head()

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,ra
1,dataset/DevanagariHandwrittenCharacterDataset/...,ra
2,dataset/DevanagariHandwrittenCharacterDataset/...,ra
3,dataset/DevanagariHandwrittenCharacterDataset/...,ra
4,dataset/DevanagariHandwrittenCharacterDataset/...,ra


In [13]:
testing_data.head()

,img_path,label
0,dataset/DevanagariHandwrittenCharacterDataset/...,ra
1,dataset/DevanagariHandwrittenCharacterDataset/...,ra
2,dataset/DevanagariHandwrittenCharacterDataset/...,ra
3,dataset/DevanagariHandwrittenCharacterDataset/...,ra
4,dataset/DevanagariHandwrittenCharacterDataset/...,ra


In [14]:
training_data["label"].unique()

array(['ra', 'chha', 'patalosaw', 'dha', 'la', 'ga', 'yaw', 'na', '3',
       'petchiryakha', 'pha', 'gya', 'da', 'kha', 'tha', '1', 'ja', '4',
       'ma', 'motosaw', 'chhya', 'ha', '6', 'taamatar', '2', 'yna',
       'adna', 'tra', 'kna', 'tabala', 'ba', 'waw', '7', 'pa', '0', 'jha',
       'bha', 'daa', '9', '8', 'dhaa', 'gha', 'cha', 'thaa', 'ka', '5'],
      dtype=object)

In [16]:
range(len(training_data['label'].unique()))

range(0, 46)

In [18]:
character2int = dict(zip(training_data["label"].unique(),range(len(training_data["label"].unique()))))

In [22]:
character2int.keys()

dict_keys(['ra', 'chha', 'patalosaw', 'dha', 'la', 'ga', 'yaw', 'na', '3', 'petchiryakha', 'pha', 'gya', 'da', 'kha', 'tha', '1', 'ja', '4', 'ma', 'motosaw', 'chhya', 'ha', '6', 'taamatar', '2', 'yna', 'adna', 'tra', 'kna', 'tabala', 'ba', 'waw', '7', 'pa', '0', 'jha', 'bha', 'daa', '9', '8', 'dhaa', 'gha', 'cha', 'thaa', 'ka', '5'])

In [23]:
training_data["label"].replace(to_replace=character2int.keys(),value=character2int.values(),
                               inplace=True)

testing_data.replace(to_replace=character2int.keys(),value=character2int.values(),
                     inplace=True)

In [31]:
Y_true_train = to_categorical(y=training_data["label"],num_classes=46)
Y_true_test = to_categorical(y=testing_data["label"],num_classes=46)

In [33]:
Y_true_test,Y_true_train

(array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32),
 array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.]], dtype=float32))

In [34]:
Y_true_train.shape

(78200, 46)

In [35]:
def multiclass_dnn():

    input_to_dnn = Input(shape=(1024,))
    first_dense_out = Dense(units=1024,activation="relu")(input_to_dnn)
    second_dense_out = Dense(units=1024,activation="relu")(first_dense_out)
    second_dense_out = BatchNormalization()(second_dense_out)
    output = Dense(units=46,activation="softmax")(second_dense_out)

    return Model(inputs=[input_to_dnn],outputs=[output])